# Using SunPy to Overlay your Photos of the Eclipse with Satellite Data

In [ ]:
import datetime

import numpy as np
import matplotlib.image
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import astropy.units as u

import scipy.ndimage as ndimage
from skimage.transform import hough_circle, hough_circle_peaks

import astropy.wcs
from astropy.coordinates import EarthLocation, SkyCoord

import sunpy.coordinates.sun
import sunpy.coordinates
from sunpy.map.header_helper import make_fitswcs_header
from astropy.time import Time
from sunpy.net import Fido, attrs as a

import exifread

We have defined a few helper functions in this `eclipse_helpers` file. To see it's contents click [here](./eclipse_helpers.py).

In [ ]:
from eclipse_helpers import solar_eclipse_image, get_camera_metadata

[enter some gumpf about the eclipse here]

In this blog post we will take an image from the 2017 solar eclipse over North America and walk through processing it with SunPy and other Python libraries so that you can generate a coordinate system for it, thereby allowing it to have other solar data overplotted and other analysis done.

## Step 1 - Read in the image and find the Sun

The first step is to read the image in and then extract critical information about the image from the metadata. To be able to do this alignment the two most important things we need to know are the GPS coordinates of where the photo was taken and the time of the image.

In [ ]:
# read in the image and flip it so that it's correct
im_rgb = np.flipud(matplotlib.image.imread(solar_eclipse_image))
# remove color info
im = np.average(im_rgb, axis=2)

In [ ]:
plt.imshow(im, origin="lower", cmap="gray")

In [ ]:
tags = exifread.process_file(open(solar_eclipse_image, 'rb'))

In [ ]:
camera_metadata = get_camera_metadata(tags)

In [ ]:
# If you are missing gps coordinates or time you will need to set it here.
# Our time on the camera is wrong, so explicitly set it.
camera_metadata["time"] = datetime.datetime(2017, 8, 21, 17, 27, 13) # don't forget to convert your time to UTC!

### Finding the Sun

The next step is to locate the edge of the Sun in the image. This allows us to find the Sun center, which is needed to align the data.

To do this we are going to use various image manipulation routines.

We start with a Gaussian blur to help segment the solar disc from the corona.

In [ ]:
blur_im = ndimage.gaussian_filter(im, 8)
mask = blur_im > blur_im.mean() * 3
plt.imshow(mask)

Next we use label the regions in the image to find the bounding box of the bright corona.

In [ ]:
label_im, nb_labels = ndimage.label(mask)

slice_x, slice_y = ndimage.find_objects(label_im==1)[0]
roi = blur_im[slice_x, slice_y]

The next step is to detect the inner edge of the bright corona, to assist with this we apply a [Sobel filter](https://en.wikipedia.org/wiki/Sobel_operator) in both the x and y directions, and then calculate a single image from the two directions.

In [ ]:
sx = ndimage.sobel(roi, axis=0, mode='constant')
sy = ndimage.sobel(roi, axis=1, mode='constant')
sob = np.hypot(sx, sy)

The next step is to use scikit-image to apply the [Hough Transform](https://en.wikipedia.org/wiki/Hough_transform) to identify circles in the image.
We then use this to extract the size in pixels of the solar disk and its center.

In [ ]:
hough_radii = np.arange(np.floor(np.mean(sob.shape)/4), np.ceil(np.mean(sob.shape)/2), 10)
hough_res = hough_circle(sob > (sob.mean() * 5), hough_radii)

# Select the most prominent circle
accums, cy, cx, radii = hough_circle_peaks(hough_res, hough_radii, total_num_peaks=1)

In this plot we demonstrate how this has worked. The first frame is the cropped original image, the middle frame is the Sobel filtered image used to apply the Hough transform and the right frame is the fitted circle on the original image.

In [ ]:
fig, ax = plt.subplots(ncols=3, nrows=1, figsize=(9.5, 6))
ax[0].imshow(im[slice_x, slice_y])
ax[0].set_title('Original')
ax[1].imshow(sob > (sob.mean() * 5))
ax[1].set_title('Derivative')
circ = Circle([cy, cx], radius=radii, facecolor='none', edgecolor='red', linewidth=2, label='Hough fit')
ax[2].imshow(im[slice_x, slice_y])
ax[2].add_patch(circ)
ax[2].set_title('Original with fit')
plt.legend()

Here we add units to the circle fit parameters, and also provide the option of fudging the result a little if needed.

In [ ]:
fudge_shift_x = 0 * u.pix # update this in case the fit needs to be shifted in x
fudge_shift_y = 0 * u.pix # update this in case the fit needs to be shifted in y
im_cx = (cx + slice_x.start) * u.pix + fudge_shift_x
im_cy = (cy + slice_y.start) * u.pix + fudge_shift_y
im_radius = radii * u.pix

We can use sunpy to calculate the angular size of the solar disk as seen from Earth at a given time.

In [ ]:
rsun_obs = sunpy.coordinates.sun.angular_radius(camera_metadata["time"])
rsun_obs

Combining this angular radius with the radius of the solar disc in pixels gives us the angular size of the pixels in the image.

In [ ]:
plate_scale = rsun_obs / im_radius
plate_scale

Finally, we can use the GPS coordinates of the image to calculate the orientation of the Sun as seen from that location on Earth. This gives us a rotation angle between our image and the solar north pole. If your camera wasn't perfectly horizontal then you may need to fudge this a little. However, in a later step we will improve the accuracy of this rotation angle by finding stars in the image.

In [ ]:
loc = EarthLocation(lat=camera_metadata["gps"][0], lon=camera_metadata["gps"][1])
fudge_angle = 0.0 * u.deg # update this in case your camera was not perfectly level.
solar_rotation_angle = sunpy.coordinates.sun.orientation(loc, camera_metadata["time"]) + fudge_angle
solar_rotation_angle

Now we have all the required coordinate information about our eclipse image, we can build a sunpy `Map` object, which is a coordinate aware 2D image.
To do this we first make a header with all this solar coordinate metadata in it.

In [ ]:
header = make_fitswcs_header(
    im,
    SkyCoord(0, 0, unit=u.arcsec, frame="helioprojective", observer=loc.get_itrs(Time(camera_metadata["time"])), obstime=camera_metadata["time"]),
    reference_pixel=u.Quantity([im_cy[0], im_cx[0]]),
    scale=np.ones(2) * plate_scale,
    rotation_angle=solar_rotation_angle,
    exposure=camera_metadata.get("exposure_time"),
    instrument=camera_metadata.get("camera_model"),
    observatory=camera_metadata.get("author"),
)

In [ ]:
eclipse_map = sunpy.map.Map(im, header)

In [ ]:
eclipse_map

## Step 2 - Finding Stars to improve the accuracy

The next thing we can do is locate a known star in the image and use that to improve our knoweldge of the rotation angle. In the case of the 2017 eclipse the very bright star (magnitude 1.4) [Regulus](https://en.wikipedia.org/wiki/Regulus) was close to the Sun.

For the 2024 eclipse there isn't such a bright star in proxmity to the Sun, which may make this method of aligning your image more challenging. The best candidate looks to be [Alpha Piscium](https://en.wikipedia.org/wiki/Alpha_Piscium) which is a binary system with a combined magnitude of 3.82, significantly dimmer.

You can see the stars close to the Sun by using [Stellarium](https://stellarium-web.org/skysource/Sun?fov=1.1092&amp;date=2024-04-08T18:30:47Z&amp;lat=28.86&amp;lng=-100.53&amp;elev=0).

As Regulus is a well known star, we can create a coordinate object for it, including it's distance.

In [ ]:
regulus = SkyCoord(ra='10h08m22.311s', dec='11d58m01.95s', distance=79.3 * u.lightyear, frame='icrs')
regulus

Using this we can then plot the expected location of Regulus over our eclipse image.
I have set the scaling on this image such that it make Regulus more visible, you can see that the expected location and the actual location are quite different.

In [ ]:
fig = plt.figure(figsize=(9,9))
ax = plt.subplot(projection=eclipse_map)
eclipse_map.plot(axes=ax, vmax=20)
ax.plot_coord(regulus, 'o', markeredgewidth=0.5, markeredgecolor='w', 
              markerfacecolor='None', markersize=10, label='Regulus')
eclipse_map.draw_grid(axes=ax)
eclipse_map.draw_limb(axes=ax)
plt.legend()
plt.show()

We can calculate the expected distance from the center of the Sun to Regulus in pixels, and we will then use this to aid the detection of Regulus in the image by filtering out all pixels closer to the Sun than this.

In [ ]:
regulus_pixel = eclipse_map.wcs.world_to_pixel(regulus) * u.pix
regulus_r = np.sqrt((regulus_pixel[0] - im_cx)**2 + (regulus_pixel[1] - im_cy)**2)
regulus_r

In [ ]:
pix_x = np.arange((eclipse_map.dimensions[0]).value) - im_cy.value
pix_y = np.arange((eclipse_map.dimensions[1]).value) - im_cx.value
xx, yy = np.meshgrid(pix_x, pix_y)
r = np.sqrt(xx**2 + yy**2)

filter_r = regulus_r - (regulus_r/5)

masked = im.copy()
masked[r < filter_r.value] = masked.min()

Having masked out most of the Sun and it's corona, we can now use the [photutils](https://photutils.readthedocs.io/en/stable/index.html) package to identify stars in the image. With some tweaking of the parameters, we make it so it only finds Regulus.

In [ ]:
from photutils.detection import DAOStarFinder

In [ ]:
daofind = DAOStarFinder(fwhm=3.0, threshold=masked.max()/2)    
sources = daofind(masked)    
print(sources)

In [ ]:
regulus_x, regulus_y = sources[0]['xcentroid'] * u.pix, sources[0]['ycentroid'] * u.pix

Having found the actual pixel coordinates of Regulus, we can compare them to our expected coordinates from our fitted coordinate system.

In [ ]:
print(regulus_pixel[0], regulus_x)
print(regulus_pixel[1], regulus_y)

We can then calculate the angular offset between our expected location and our fitted location and then add use this difference to correct our `solar_rotation_angle`.

In [ ]:
vec_image = np.array([regulus_x - im_cx, regulus_y - im_cy]).T
vec_coord = np.array([regulus_pixel[0] - im_cx, regulus_pixel[1] - im_cy])
fudge_angle = np.arccos((vec_image @ vec_coord) / (np.linalg.norm(vec_image) * np.linalg.norm(vec_coord)))*u.rad
fudge_angle = fudge_angle.flat[0] * -1
fudge_angle.to(u.deg)

In [ ]:
header = make_fitswcs_header(
    im,
    SkyCoord(0, 0, unit=u.arcsec, frame="helioprojective", observer=loc.get_itrs(Time(camera_metadata["time"])), obstime=camera_metadata["time"]),
    reference_pixel=u.Quantity([im_cy[0], im_cx[0]]),
    scale=np.ones(2) * plate_scale,
    rotation_angle=solar_rotation_angle + fudge_angle,
    exposure=camera_metadata["exposure_time"],
    instrument=camera_metadata["camera_model"],
)

In [ ]:
eclipse_map = sunpy.map.Map(im, header)

We can now see that the Regulus is roughly where we expect it to be.

In [ ]:
fig = plt.figure(figsize=(9,9))
ax = plt.subplot(projection=eclipse_map)
eclipse_map.plot(axes=ax, vmax=20)
ax.plot_coord(regulus, 'o', markeredgewidth=0.5, markeredgecolor='w', 
              markerfacecolor='None', markersize=10, label='Regulus')
eclipse_map.draw_grid(axes=ax)
eclipse_map.draw_limb(axes=ax)
plt.legend()
plt.show()

## Step 3 - Combine your image with Satellite data

For this final flourish, we use SunPy to download a 17.1 nm image from the AIA instrument on the [SDO](https://en.wikipedia.org/wiki/Solar_Dynamics_Observatory) satellite.

In [ ]:
# Replace the time below with the time in UT of the eclipse
t = a.Time('2017-08-21 17:27:00', "2017-08-21 17:45:00", eclipse_map.date)
aia_result = Fido.search(t, a.Instrument('AIA'), a.Wavelength(171*u.Angstrom))
aia_result

In [ ]:
files = Fido.fetch(aia_result[0,0])

Having downloaded this data we create a sunpy Map from it, and then downsample it in resolution to make things faster.

In [ ]:
aia_map = sunpy.map.Map(files[0])
aia_map = aia_map.superpixel((4, 4)*u.pix)  # Remove this line to use the full resolution version.

Before we can plot the data together we must align the images, we do this using the `reproject_to` method.

In [ ]:
aligned_aia = aia_map.reproject_to(eclipse_map.wcs)

In [ ]:
fig = plt.figure(figsize=(9,9))
ax = plt.subplot(projection=eclipse_map)
eclipse_map.plot(axes=ax)
eclipse_map.draw_grid(axes=ax)
aligned_aia.plot(axes=ax)